In [117]:
import pandas as pd
from bokeh.io import curdoc, output_file, output_notebook
from bokeh.plotting import reset_output, figure, show
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.models import CategoricalColorMapper
from bokeh.palettes import Spectral6
from bokeh.layouts import widgetbox, row, gridplot
from bokeh.models import Slider, Select

In [118]:
data = pd.read_csv("national-history.csv")
data.head()

,date,death,deathIncrease,inIcuCumulative,inIcuCurrently,hospitalizedIncrease,hospitalizedCurrently,hospitalizedCumulative,negative,negativeIncrease,onVentilatorCumulative,onVentilatorCurrently,positive,positiveIncrease,states,totalTestResults,totalTestResultsIncrease
0,2021-03-07,515151.0,842,45475.0,8134.0,726,40199.0,776361.0,74582825.0,131835,4281.0,2802.0,28756489.0,41835,56,363825123,1170059
1,2021-03-06,514309.0,1680,45453.0,8409.0,503,41401.0,775635.0,74450990.0,143835,4280.0,2811.0,28714654.0,60015,56,362655064,1430992
2,2021-03-05,512629.0,2221,45373.0,8634.0,2781,42541.0,775132.0,74307155.0,271917,4275.0,2889.0,28654639.0,68787,56,361224072,1744417
3,2021-03-04,510408.0,1743,45293.0,8970.0,1530,44172.0,772351.0,74035238.0,177957,4267.0,2973.0,28585852.0,65487,56,359479655,1590984
4,2021-03-03,508665.0,2449,45214.0,9359.0,2172,45462.0,770821.0,73857281.0,267001,4260.0,3094.0,28520365.0,66836,56,357888671,1406795


In [119]:
# Make a list of the unique values from the region column: regions_list
factors = data.date.unique().tolist()

# Make a color mapper: color_mapper
colors = ["red","green","blue","black","orange","brown","grey","purple","yellow","cyan","pink","peru"]
color_mapper = CategoricalColorMapper(factors=factors, palette=Spectral6)

In [120]:
# Make the ColumnDataSource: source
source = ColumnDataSource(data={
    "Date": data.loc[[0]].date,
    "Death": data.loc[[0]].death,
    "DeathIncrease" : data.loc[[0]].deathIncrease
})

In [121]:
# hover tool
hover = HoverTool(tooltips = [("Date","@Date"),("Death","@Death"),("Death Increase","@DeathIncrease")])

# plotting
plot=figure(title ="Covid-19 Death Rate",tools=[hover,"crosshair","pan","box_zoom"])
plot.circle("Date","Death",source=source,color=dict(field="DeathIncrease", transform=color_mapper), legend_label='DeathIncrease',hover_color ="red")

# Set the legend and axis attributes
plot.legend.location = 'bottom_left'

# Define the callback function: update_plot
def update_plot(attr, old, new):
    # set the `dt` name to `slider.value` and `source.data = new_data`
    dt = slider.value
    date = date_select.value
    death = death_select.value
    # Label axes of plot
    plot.xaxis.axis_label = date
    plot.yaxis.axis_label = death
    # new data
    new_data = {
        'Date'        : data.loc[date,c1].date,
        'Death'       : data.loc[date,c2].death,
        "deathIncrease" : data.loc[date].deathIncrease
    }
    source.data = new_data

# Make a slider object: slider
slider = Slider(start=1970, end=2010, step=1, value=1970, title='Year')
slider.on_change('value',update)

# Make dropdown menu for x and y axis
# Create a dropdown Select widget for the x data: x_select
x_select = Select(
    options=['deathIncrease', 'hospitalizedIncrease', 'negativeIncrease', 'positiveIncrease'],
    value='date',
    title='Date_axis'
)
# Attach the update_plot callback to the 'value' property of x_select
x_select.on_change('value', update)

# Create a dropdown Select widget for the y data: y_select
y_select = Select(
    options=['deathIncrease', 'hospitalizedIncrease', 'negativeIncrease', 'positiveIncrease'],
    value='death',
    title='Death_axis'
)
# Attach the update_plot callback to the 'value' property of y_select
y_select.on_change('value', update)

show(plot)